# Imports

In [1]:
%%capture
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
from tqdm.auto import tqdm
from time import sleep
import scienceplots
import subprocess
pd.set_option('display.max_columns', None)
plt.style.use(['default','notebook']) #plt.style.use(['science','notebook'])
plt.tight_layout()
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None


In [2]:
# import functions from previous 
from OSPE_Analysis_funcs import *

<Figure size 800x600 with 0 Axes>

In [ ]:
#install cosmic
!pip3 install cosmic-popsynth #(can do this on unix command-line as well, I used Jupyter Notebook)

In [3]:
#import cosmic stuff
from cosmic import _evolvebin
from cosmic.sample.initialbinarytable import InitialBinaryTable
from cosmic.evolve import Evolve

#import pandas
import pandas as pd



# Useful Functions

In [4]:

## FUNCTIONS
pi = np.pi
k2 = 4*pi*pi
MS_Types=[0.,1.]
WD_Types=[10.,11.,12.]
MSWD_Types=[0.,1.,10.,11.,12.]

cols =  [
        'N','t','t_new','sur','m1','m1_new','m2','m2_new','m3','m3_new',
        'e1','e1_new','e2','e2_new','a1','a1_new','a2','a2_new','i','i_new','P_in','P_in_new',
        'type1','type2','type3','type1_new','type2_new','type3_new',
        'G1o', 'G1n', 'G2o', 'G2n', 'Gtoto', 'Gtotn', 'cosi_o', 'cosi_n', 'needs_Re', 'Ad_or_Kick'
        ]


def epsilon(a1,a2,e2):
    return (   (a1/a2)* (  e2/(1-e2**2)  )  )

def get_tidal_locked_condition(row):
    a1,Roche1,e1,P1 = row['a1'],row['Roche1'],row['e1'],row['P_in']
    beta,spintot,spin2h = row['beta'],row['spintot'],row['spin2h']
    M_PI = np.pi
    
    tl_crit1 = (a1/Roche1 <= 5 or a1 <= 0.1) 
    tl_crit2 = ( e1 <= 1e-4 )
    tl_crit3 = (a1 <= 0.1 and e1 <= 1e-2)

    tl_condition = ( e1 <= 1e-2 and tl_crit1 and tl_crit2) or tl_crit3
    return tl_condition

def get_t_ekl(m1,m2,m3,e2,P1,P2):
    """
    Characteristic quadrupole timescale fro EKL (eq 27 from review)
    """
    return (8./(15*np.pi))*((m1+m2+m3)/m3)*(P2*P2/P1)*np.sqrt(1-e2*e2)*(1-e2*e2)

# can also use this for a2 if inner binary is in c=
def get_a_new(m1,m2,m2n,e1,EA1,a_old, uk=0,theta=0): #(10) in Lu+2019

    B = (m1+m2n) / (m1+m2)
    a1_ratio = B * (1-e1*np.cos(EA1)) / (  2*B - (1+e1*np.cos(EA1))* (1+uk*uk + 2*uk*np.cos(theta))     )
    return a1_ratio * a_old
"""
"""
def a2_new(m1,m2n,m3,a2,e2,E2):
    M = m1 + m2n + m3
    R = a2( 1 - e2*np.cos(E2) )
    
    fv = k2 * M * (2/R  - 1/a2)
    inv = 2 / (a2*(1-e2*np.cos(E2)))   -  fv*fv / ( k2* (m1+m2n + m3) )
    a2n = 1/inv
    
    return a2n

def get_e1_new(r,vr,a1n,m1,m2n,theta=90): #(19) from Lu et al.
    
    e_squared = 1 - np.power(r*vr*np.sin(theta), 2) / (a1n* k2 * (m1+m2n))
    
    return np.sqrt(e_squared)

def get_e2_new(m1,m2n,m3,a2n,R,V2n, theta=90):
    #theta = np.random.uniform(0,np.pi)
    M = m1 + m2n + m3
    term = np.power(a2n*k2*M  ,-1) * np.power(R*V2n*np.sin(theta),2)
    return 1 - term

def get_i(G1,G2,Gtot):
    cosi = (Gtot**2 - G1**2 - G2**2 ) / (2*G1*G2)
    
    return np.arccos(cosi)*180/pi #in degrees

def get_L_rigid(M, R, spin, shape = 'sphere'):
    if shape == 'sphere': 
        I = (2/5) * M * R**2
    w = 2*pi*spin
    return I*w
    
def get_L1(m1,m2,a1):
    mu = (m1*m2 / (m1+m2) )
    L1 = mu * np.sqrt(k2 * (m1+m2)*a1)
    return L1

def get_h(m1,m2,a,e):
    
    hsqr = k2*(m1+m2)*a * np.sqrt(1-e*e)
    return np.sqrt(hsqr)

def get_G_from_h(m1,m2,h):
    mu = m1*m2 / (m1+m2)
    return mu*h

def get_L2(m1,m2,m3,a2):
    M = m1 + m2 + m3
    mu = m3*(m1+m2) / M
    L2 = mu * np.sqrt(k2*M*a2)
    return L2

def get_G(L,e):
    
    return L * np.sqrt(1-e*e)

def i_from_c(old_masses, new_masses,e1o,e1n,old_Gs,new_Gs):
    
    m1o,m2o,m3o = old_masses
    m1n,m2n,m3n = new_masses
    G1o, G2o = old_Gs
    G1n, G2n = new_Gs
    
    mu1o = (m1o * m2o) / (m1o+m2o)
    mu1n = (m1n * m2n) / (m1n+m2n)

    mu2o = m3o*(m1o + m2o) / (m1o+m2o+m3o) #mu2o
    mu2n = m3n*(m1n + m2n) / (m1n+m2n+m3n) #mu2n

    factor1 = np.sqrt(   (1-e1n*e1n) / (1-e1o*e1o) )
    alpha1 = (mu1n/mu1o) * G1o * factor1
    alpha2 = (mu2n / mu2o) * G2o
    
    cosi_n = 0.5 * (    ( alpha1**2 + alpha2**2 - (G1n**2 + G2n**2) ) / (G1n*G2n - alpha1*alpha2)  )
    
    return np.arccos(cosi_n) *180/pi
    
def new_Gtot_from_old(initial_masses : tuple, new_masses : tuple, G10 : float, G20 : float, i0 : float,
                      e1_old : float, e1_new : float) -> float:
    """Get Gtot new from the old G's

    Args:
        initial_masses (tuple): m1o, m2o, m2o
        new_masses (tuple): m1n, m2n, m3n
        G10 (float): old G1
        G20 (float): old G2
        i0 (float): Old total inclination
        old_eccs (float): e1o,e2o
        new_eccs (float): e1n,e2n
    Returns:
        _type_: new G
    """
    
    m1o, m2o, m3o = initial_masses 
    m1n, m2n, m3n = new_masses
    
    e1o = e1_old
    e1n = e1_new
    
    mu1o = (m1o * m2o) / (m1o+m2o)
    mu1n = (m1n * m2n) / (m1n+m2n)
    
    mu2o = m3o*(m1o + m2o) / (m1o+m2o+m3o) #mu2o
    mu2n = m3n*(m1n + m2n) / (m1n+m2n+m3n) #mu2n
    
    factor1 = np.sqrt( (1-e1n*e1n) / (1-e1o*e1o)     )
    G10_u = G10*factor1

    Gsqr = ( (  (mu1n/mu1o)**2  ) * G10_u**2  )  + ( (  (mu2n/mu2o)**2  ) * G20**2   ) + 2*(mu1n/mu1o)*(mu2n/mu2o) * G10_u*G20*np.cos(i0)
    
    return np.sqrt(Gsqr)

def new_i_from_stuff(initial_masses : tuple, new_masses : tuple, i0 : float,
                      e1_old : float, e1_new : float, e2_old : float,  
                      a1n : float, a2 : float) -> float:
    
    m1 , m2 , m3  = initial_masses 
    m1n, m2n, m3n = new_masses
    
    e1o = e1_old
    e2o = e2_old
    e1n = e1_new
    e2n = e2_old
    #mu1  = (m1  * m2 ) / (m1 +m2 )
    mu1n = (m1n * m2n) / (m1n + m2n)
    
    mu2  = m3 *(m1  + m2 ) / (m1 +m2 +m3 ) #mu2o
    mu2n = m3n*(m1n + m2n) / (m1n+m2n+m3n) #mu2n
    
    L1 = mu1n * np.sqrt(k2*(m1+m2)*a1n)
    L2 = mu2n * np.sqrt(k2*(m1+m2+m3)*a2)
    
    G1n   =  mu1n*np.sqrt((m1n + m2n) * a1n * (1-e1n**2))       #L1 * np.sqrt(1-e1n**2) #mu1n*np.sqrt((m1n + m2n) * a1n * (1-e1n**2)) 
    G2    =  mu2 *np.sqrt((m3 * (m1 + m2) * a2 * (1-e1o**2)))   #L2 * np.sqrt(1-e2n**2) #mu2 *np.sqrt((m3 * (m1 + m2) * a2  * (1-e1o**2))) 
    
    G2n=(mu2n/mu2)*G2
    Gtotn2 = G1n**2 + G2n**2 + 2* G1n * G2n * np.cos(i0*pi/180) 
    
    cosin=(Gtotn2-G1n**2-G2n**2)/(2.*G1n*G2n)
    
    return np.arccos(cosin)*180/pi
    
def get_r_mag(a,ecc,EA):
    
    return a*(1 - ecc*np.cos(EA))

def get_vr(m1,m2,r,a):
    mu = k2*(m1+m2)
    return np.sqrt( mu* (2/r - 1/a)    )
# af = (mi/mf)ai, mi and mf are total mass of inner binary
def get_a_adiabatic(initial_masses : tuple ,final_masses : tuple ,ai :float):
    
    Mi = sum(initial_masses)
    Mf = sum(final_masses)
    
    return ( (Mi/Mf) * ai )
    
def sample_angle(sampler='uniform'): #Sample eccentric anaomly
    if sampler == 'uniform':
        return np.random.uniform(0,np.pi)
    
def get_r_avg(a,e):
    
    return a*(1+0.5* np.power(e,2)   ) #time averages separation in two body orbit

def get_kick_params():
    #WRONG
    
    EA1 = sample_angle(m1,m2,m3,e2_old,a2_old,a1,e1)
    new_a2 = get_a_new(m1 = m3,m2 = m1+m2, m2n = m1, 
                                    e1=e2_old,EA1=EA1,a_old = a2_old) #m2_new = m3


    R = get_r_avg(a2_old,e2_old) #average inner separation before merger
    r = get_r_avg(a1,e1) #average inner separation before merger
    vr = get_vr(m1+m2,m3,R,a2_old)

    new_e2 = get_e1_new(R,vr,new_a2,m1+m2,m3)

    L1n, L2n = get_L1(m1,m2,a1), get_L2(m1,m2,m3,new_a2)
    G1n, G2n = get_G(L1n,e1), get_G(L2n,new_e2)

    Gtot = G1n*np.cos(np.deg2rad(i1)) + G2n*np.cos(np.deg2rad(i2))
    new_i = get_i(G1n,G2n,Gtot) 

    if new_i == np.nan: print(G1n,G2n,Gtot)


# Examples/Testing

In [1434]:
## Multiple Binaries
#Initial Conditions
tidally_locked = tidally_locked[(tidally_locked.t > 1e6)]

m1 = tidally_locked.m1.to_list() #[2.0, 2.0]
m2 = tidally_locked.m2.to_list() #[1.0, 1.0]
porb = tidally_locked.P_in.to_list() #[1, 500]
ecc = tidally_locked.e1.to_list() #[0.1, 0.1]
tphysf = [12500] * len(tidally_locked)#((12.5e9 - tidally_locked.t) / 1e6 ).tolist() #((12.5e9 - tidally_locked.t) / 1e6 ).tolist() #[12.5e3] * len(tidally_locked)
tphys = (tidally_locked.t / 1e6).tolist()
kstar1 = tidally_locked.type1.to_list() #[1, 1]
kstar2 =  tidally_locked.type2.to_list() #[1, 1]
rad1,rad2 = (tidally_locked.R1 * 215.032).to_list(), (tidally_locked.R2 * 215.032).to_list() #R_sun
omega_spin1 = tidally_locked.spintot.tolist()
omega_spin2 = np.sqrt(tidally_locked.spin2h**2.+tidally_locked.spin2e**2.+tidally_locked.spin2q**2.) .tolist()
metallicity = [0.02]*len(tidally_locked)
tacc1 = tacc2 = tidally_locked.t.tolist()
lum1 = [75]*len(tidally_locked)
lum2 = [1]*len(tidally_locked)
massc_1=[0.5]*len(tidally_locked)
massc_2=[0.1]*len(tidally_locked)
radc_1=[0.001]*len(tidally_locked)
radc_2=[0.25]*len(tidally_locked)
renv_1 = [a_i - b_i for a_i, b_i in zip(rad1, radc_1)]
renv_2 = [a_i - b_i for a_i, b_i in zip(rad2, radc_2)]
menv_1 = [a_i - b_i for a_i, b_i in zip(m1, massc_1)]
menv_2 = [a_i - b_i for a_i, b_i in zip(m2, massc_2)]

binary_set = InitialBinaryTable.InitialBinaries(m1 = m1, m2 = m2, porb=porb, ecc=ecc, tphysf=tphysf,
                                                kstar1=kstar1, kstar2=kstar2, metallicity = metallicity, 
                                                rad_1=rad1, rad_2=rad2, ospin_1 = omega_spin1, ospin_2 = omega_spin2,
                                                lumin_1=lum1,lumin_2=lum2,massc_1=massc_1,massc_2=massc_2,radc_1=radc_1,
                                                 radc_2=radc_2, renv_1=renv_1,renv_2 = renv_2, menv_1 = menv_1, menv_2 = menv_2,
                                                 tphys=tphys)
binary_set.head(5)                                    


,kstar_1,kstar_2,mass_1,mass_2,porb,ecc,metallicity,tphysf,mass0_1,mass0_2,rad_1,rad_2,lum_1,lum_2,massc_1,massc_2,radc_1,radc_2,menv_1,menv_2,renv_1,renv_2,omega_spin_1,omega_spin_2,B_1,B_2,bacc_1,bacc_2,tacc_1,tacc_2,epoch_1,epoch_2,tms_1,tms_2,bhspin_1,bhspin_2,tphys,binfrac
0,5.0,1.0,4.426909,1.862399,12562.389429,0.000026,0.02,12500.0,4.426909,1.862399,1116.799012,1.584723,75.0,1.0,0.5,0.1,0.001,0.25,3.926909,1.762399,1116.798012,1.334723,0.171888,3109.866968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.531250,1.0
1,5.0,1.0,3.207298,2.747198,59.473209,0.000408,0.02,12500.0,3.207298,2.747198,23.088889,2.942169,75.0,1.0,0.5,0.1,0.001,0.25,2.707298,2.647198,23.087889,2.692169,38.549250,164.211276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,318.719846,1.0
2,5.0,1.0,3.345706,1.018200,871.468437,0.000001,0.02,12500.0,3.345706,1.018200,198.570118,0.913153,75.0,1.0,0.5,0.1,0.001,0.25,2.845706,0.918200,198.569118,0.663153,2.623466,344.048550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,335.593750,1.0
3,5.0,1.0,3.107238,1.876399,1137.763752,0.000062,0.02,12500.0,3.107238,1.876399,195.135777,1.787043,75.0,1.0,0.5,0.1,0.001,0.25,2.607238,1.776399,195.134777,1.537043,2.012181,2705.178929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,412.602084,1.0
4,5.0,1.0,4.110897,3.935597,89.642311,0.000088,0.02,12500.0,4.110897,3.935597,39.388508,5.024758,75.0,1.0,0.5,0.1,0.001,0.25,3.610897,3.835597,39.387508,4.774758,25.573680,1367.557235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,168.226231,1.0


In [1429]:
## TEST ##

"""
## Multiple Binaries
#Initial Conditions
m1 = 4.362826
m2 = 4.362826
porb = 1000
ecc = 0.1
tphysf = 12500
tphys=1000
tms_1,tms_2 = 142.393315, 142.393315

kstar1 = 5.
kstar2 =  5.
rad1,rad2 = 84.364498, 84.364498
omega_spin1, omega_spin2 = 0.000230, 0.000230

metallicity = 0.02
tacc1 = tacc2 = tidally_locked.t.tolist()
lum1,lum2 = 1197.929529	, 1197.929529

massc_1, massc_2 =0.992213,	0.992213

radc_1,radc_2 =0.203407,	0.203407

renv_1,renv_2 = 84.161091,	1.

menv_1,menv_2 = 3.370612, 3.370612
aj_1, aj_2 = 167.127985,	167.127985
teff_1,	teff_2 = 3721.172608,	3721.172608

binary_set = InitialBinaryTable.InitialBinaries(m1 = m1, m2 = m2, porb=porb, ecc=ecc, tphysf=tphysf,
                                                kstar1=kstar1, kstar2=kstar2, metallicity = metallicity, 
                                                rad_1=rad1, rad_2=rad2, ospin_1 = omega_spin1, ospin_2 = omega_spin2,
                                                lumin_1=lum1,lumin_2=lum2,massc_1=massc_1,massc_2=massc_2,radc_1=radc_1,
                                                 radc_2=radc_2, renv_1=renv_1,renv_2 = renv_2, menv_1 = menv_1, menv_2 = menv_2,
                                                 aj_1=aj_1,aj_2=aj_2,tms_1=tms_1,tms_2=tms_2,teff_1=teff_1,teff_2=teff_2,
                                                 tphys=tphys)
binary_set      
"""                                 


,kstar_1,kstar_2,mass_1,mass_2,porb,ecc,metallicity,tphysf,mass0_1,mass0_2,rad_1,rad_2,lum_1,lum_2,massc_1,massc_2,radc_1,radc_2,menv_1,menv_2,renv_1,renv_2,omega_spin_1,omega_spin_2,B_1,B_2,bacc_1,bacc_2,tacc_1,tacc_2,epoch_1,epoch_2,tms_1,tms_2,bhspin_1,bhspin_2,tphys,binfrac
0,5.0,5.0,4.362826,4.362826,1000.0,0.1,0.02,12500.0,4.362826,4.362826,84.364498,84.364498,1197.929529,1197.929529,0.992213,0.992213,0.203407,0.203407,3.370612,3.370612,84.161091,1.0,0.00023,0.00023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,142.393315,142.393315,0.0,0.0,1000.0,1.0


In [1432]:
%%capture
timestep_conditions = [['RRLO_1>=1', 'dtp=10.0'], ['RRLO_2>=1', 'dtp=10.0'],
                       ['evol_type==2', 'dtp=10.0'], ['sep>=22.','ecc>0', 'dtp=1.'],
                       [['binstate==1', 'dtp=10.']]  ]
#timestep_conditions = [['evol_type=2', 'dtp=0.0']]
BSEDict = {'xi': 1.0, 'bhflag': 1, 'neta': 0.5, 'windflag': 3, 'wdflag': 1, 'alpha1': 1.0, 'pts1': 0.001, 'pts3': 0.02, 'pts2': 0.01, 'epsnov': 0.001, 'hewind': 0.5, 'ck': 1000, 'bwind': 0.0, 'lambdaf': 0.0, 'mxns': 3.0, 'beta': -1.0, 'tflag': 1, 'acc2': 1.5, 'grflag' : 1, 'remnantflag': 4, 'ceflag': 0, 'eddfac': 1.0, 'ifflag': 0, 'bconst': 3000, 'sigma': 265.0, 'gamma': -2.0, 'pisn': 45.0, 'natal_kick_array' : [[-100.0,-100.0,-100.0,-100.0,0.0], [-100.0,-100.0,-100.0,-100.0,0.0]], 'bhsigmafrac' : 1.0, 'polar_kick_angle' : 90, 'qcrit_array' : [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], 'cekickflag' : 2, 'cehestarflag' : 0, 'cemergeflag' : 0, 'ecsn' : 2.25, 'ecsn_mlow' : 1.6, 'aic' : 1, 'ussn' : 0, 'sigmadiv' :-20.0, 'qcflag' : 1, 'eddlimflag' : 0, 'fprimc_array' : [2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0], 'bhspinflag' : 0, 'bhspinmag' : 0.0, 'rejuv_fac' : 1.0, 'rejuvflag' : 0, 'htpmb' : 1, 'ST_cr' : 1, 'ST_tide' : 1, 'bdecayfac' : 1, 'rembar_massloss' : 0.5, 'kickflag' : 0, 'zsun' : 0.014, 'bhms_coll_flag' : 0, 'don_lim' : -1, 'acc_lim' : -1}

bpp, bcm, initC, kick_info = Evolve.evolve(initialbinarytable=binary_set, BSEDict=BSEDict)#, timestep_conditions = timestep_conditions
                                           


In [3730]:
binary_set = InitialBinaryTable.InitialBinaries(m1 = m1, m2 = m2, porb=porb, ecc=ecc, tphysf=tphysf,
                                                kstar1=kstar1, kstar2=kstar2, metallicity = metallicity, 
                                                rad_1=rad1, rad_2=rad2, ospin_1 = omega_spin1, ospin_2 = omega_spin2,
                                                tphys=tphys
                                                )


m1, m2 =1.1718,	0.566500	
porb = np.power( ( (0.028051**3)/(m1+m2) ) ,1./2.)
ecc = 8.064448e-12
tphysf = 12500.
tphys = 3.717056e+09 / 1e6
kstar1, kstar2 = 11.0,11.0
rad1,rad2 = 0.000028* 215.032,	0.006733* 215.032
omega_spin1 = np.sqrt(3319.9900**2.+4117.942**2.+14098.9100**2.)
omega_spin2 = np.sqrt(1763.302000**2.+0.000003**2.+ (-9.888369e-07)**2.) 
metallicity = 0.02

single_binary = InitialBinaryTable.InitialBinaries(m1 = m1, m2 = m2, porb=porb, ecc=ecc, tphysf=tphysf,
                                                    kstar1=kstar1, kstar2=kstar2, metallicity = metallicity, 
                                                    rad_1=rad1, rad_2=rad2, ospin_1 = omega_spin1, ospin_2 = omega_spin2,
                                                    tphys=tphys)
BSEDict = {'xi': 1.0, 'bhflag': 1, 'neta': 0.5, 'windflag': 3, 'wdflag': 1, 'alpha1': 1.0, 'pts1': 0.001, 'pts3': 0.02, 'pts2': 0.01, 'epsnov': 0.001, 'hewind': 0.5, 'ck': 1000, 'bwind': 0.0, 'lambdaf': 0.0, 'mxns': 3.0, 'beta': -1.0, 'tflag': 1, 'acc2': 1.5, 'remnantflag': 3, 'ceflag': 0, 'eddfac': 1.0, 'ifflag': 0, 'bconst': 3000, 'sigma': 265.0, 'gamma': -2.0, 'pisn': 45.0, 'natal_kick_array' : [[-100.0,-100.0,-100.0,-100.0,0.0], [-100.0,-100.0,-100.0,-100.0,0.0]], 'bhsigmafrac' : 1.0, 'polar_kick_angle' : 90, 'qcrit_array' : [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], 'cekickflag' : 2, 'cehestarflag' : 0, 'cemergeflag' : 0, 'ecsn' : 2.5, 'ecsn_mlow' : 1.4, 'aic' : 1, 'ussn' : 0, 'sigmadiv' :-20.0, 'qcflag' : 1, 'eddlimflag' : 0, 'fprimc_array' : [2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0], 'bhspinflag' : 0, 'bhspinmag' : 0.0, 'rejuv_fac' : 1.0, 'rejuvflag' : 0, 'htpmb' : 1, 'ST_cr' : 1, 'ST_tide' : 0, 'bdecayfac' : 1, 'randomseed' : -1235453, 'grflag' : 1, 'rembar_massloss' : 0.5, 'kickflag' : 0, 'zsun' : 0.014,  'grflag' : 1, 'bhms_coll_flag' : 0, 'don_lim' : -1, 'acc_lim' : -1}

bpp, bcm, initC, kick_info = Evolve.evolve(initialbinarytable=single_binary, BSEDict=BSEDict, timestep_conditions = [['RRLO_1<1','dtp=0.0']])


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cosmic/evolve.py:258: UserWarning: The value for randomseed in initial binary table is being overwritten by the value of randomseed from either the params file or the BSEDict.
  warnings.warn("The value for {0} in initial binary table is being "


In [ ]:
m1_old,m2_old,m3_old = 1.1718,0.5665,1.3414
m1_new,m2_new,m3_new = bcm.mass_1,bcm.mass_2,
i_old = 136.0574
e1_old, e2_old = 8.064448e-12,	0.893977
e1 = bcm.ecc
a1 = np.power( ( (bcm.porb**2)*(m1+m2) ) ,1./3.)
new_a2 = (m1_new+m2_new+m3_new)/(m1_old+m2_old+m3_old)
my_i_n = new_Gtot_from_stuff ((m1_old,m2_old,m3_old),(m1,m2,m3),
                                              i_old*180/pi,e1_old,e1,e2_old, a1, new_a2 )


# +-+-+-+-+-+-+

# **START HERE **

# Read in DFs

In [5]:
#Make sure that they were written out from OSPE_Analysis files
pickle_PATH = '/Users/bijan1339/Desktop/Research/Dataframes/'

final0 = pd.read_pickle(pickle_PATH + 'final_1T_new.pkl')  
final1 = pd.read_pickle(pickle_PATH + 'final_KT_new.pkl')  
final4 = pd.read_pickle(pickle_PATH + 'final_4_old.pkl')  
final6 = pd.read_pickle(pickle_PATH + 'final_KT_old.pkl')  
final7 = pd.read_pickle(pickle_PATH + 'final_1T_old.pkl')  
final8 = pd.read_pickle(pickle_PATH + 'final_1B.pkl') 


# Define Tidally Locked Dfs

Pipeline Starts Here:
- run the following cell and all others below it in succession

In [6]:
## Get Tidal Locked Binaries
acc_sur = [0.,4.,7.]
MS_Types = [0.,1.,2.]

###########################
#DEFINE DF OF INTEREST    #
df_of_choice = final4     #
model_name = '4'         #
###########################
def initialize_tl_df(final_df):
    add_tekl_col(final_df) #add time columns
    final = final_df[ (final_df.t < 12.5e9) ] #filter so that none that finished
    tidally_locked = final[ (  final.sur.isin(acc_sur)            ) |
                            ( (final.a1<0.2) & (final.e1 < 0.05)  ) | 
                            (  final.t_quad > (12.5e9 - final.t)  ) |
                            (final.e1 > 0.80) | 
                            ( (final.t > 8e9) & (final.t < 10e9)  )
                            ]
                           
    return tidally_locked
#(1) didnt finish (2) have e1<0.05 and (3) a1 < 0.2

tidally_locked = initialize_tl_df(df_of_choice)

# Evolving Binaries in COSMIC

In [7]:
def get_COSMIC_params(tl_df ):
    N = tl_df.N.to_list()
    m1, m2, m3 = tl_df.m1.to_list(), tl_df.m2.to_list(), tl_df.m3.to_list() 
    porb = tl_df.P_in.to_list() #[1, 500]
    ecc = tl_df.e1.to_list() #[0.1, 0.1]
    tphysf = [12500.]*len(tl_df) #((12.5e9 - tl_df.t) / 1e6 ).tolist() #[12500]*len(tl_df) #((12.5e9 - tl_df.t) / 1e6 ).tolist() #((12.5e9 - tl_df.t) / 1e6 ).tolist() #[12.5e3] * len(tl_df)
    tphys = (tl_df.t / 1e6).tolist()
    kstar1, kstar2, kstar3 = tl_df.type1.to_list(), tl_df.type2.to_list(), tl_df.type3.to_list()
    rad1,rad2 = (tl_df.R1 * 215.032).to_list(), (tl_df.R2 * 215.032).to_list() #R_sun
    omega_spin1 = tl_df.spintot.tolist()
    omega_spin2 = np.sqrt(tl_df.spin2h**2.+tl_df.spin2e**2.+tl_df.spin2q**2.) .tolist()
    metallicity = [0.02]*len(tl_df)
    tacc1 = tacc2 = tl_df.t.tolist()
    
    return (N,m1,m2,m3,porb,ecc,tphysf,tphys,kstar1,kstar2,kstar3,rad1,rad2,omega_spin1,omega_spin2,metallicity,tacc1,tacc2)

    

## NO Restart

In [8]:
%%capture
tl_no_restart = tidally_locked[(tidally_locked['t_quad'] > tidally_locked['t_remain']) ] 
                               #( tidally_locked.type1.isin(MSWD_Types))  & (tidally_locked.type2.isin(MSWD_Types) )] #tquad > remaining time
N_no_restart = tl_no_restart['N'].tolist()
N,m1,m2,m3,porb,ecc,tphysf,tphys,kstar1,kstar2,kstar3,rad1,rad2,omega_spin1,omega_spin2,metallicity,tacc1,tacc2 = get_COSMIC_params(tl_no_restart)

binary_set = InitialBinaryTable.InitialBinaries(m1 = m1, m2 = m2, porb=porb, ecc=ecc, tphysf=tphysf,
                                                kstar1=kstar1, kstar2=kstar2, metallicity = metallicity, 
                                                rad_1=rad1, rad_2=rad2, ospin_1 = omega_spin1, ospin_2 = omega_spin2,
                                                tphys=tphys
                                                )

timestep_conditions = [['RRLO_1>=1', 'dtp=10.0'], ['RRLO_2>=1', 'dtp=10.0'], ['binstate==0', 'dtp=100.0'],
                       ['sep<=22.','ecc>=0', 'dtp=100.'], ['kstar_1==15.','dtp=300.'], ['kstar_2==15.','dtp=300.'],
                       ['binstate==1', 'dtp=150.0'], ['RRLO_1<1','RRLO_2<1', 'dtp=100.0']  
                       ]
#timestep_conditions = [['t>=1', 'dtp=0.0']]

BSEDict = {'xi': 1.0, 'bhflag': 1, 'neta': 0.5, 'windflag': 3, 'wdflag': 1, 'alpha1': 1.0, 'pts1': 0.001, 'pts3': 0.02, 'pts2': 0.01, 'epsnov': 0.001, 'hewind': 0.5, 'ck': 1000, 'bwind': 0.0, 'lambdaf': 0.0, 'mxns': 3.0, 'beta': -1.0, 'tflag': 1, 'acc2': 1.5, 'grflag' : 1, 'remnantflag': 4, 'ceflag': 0, 'eddfac': 1.0, 'ifflag': 0, 'bconst': 3000, 'sigma': 265.0, 'gamma': -2.0, 'pisn': 45.0, 'natal_kick_array' : [[-100.0,-100.0,-100.0,-100.0,0.0], [-100.0,-100.0,-100.0,-100.0,0.0]], 'bhsigmafrac' : 1.0, 'polar_kick_angle' : 90, 'qcrit_array' : [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], 'cekickflag' : 2, 'cehestarflag' : 0, 'cemergeflag' : 0, 'ecsn' : 2.25, 'ecsn_mlow' : 1.6, 'aic' : 1, 'ussn' : 0, 'sigmadiv' :-20.0, 'qcflag' : 1, 'eddlimflag' : 0, 'fprimc_array' : [2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0], 'bhspinflag' : 0, 'bhspinmag' : 0.0, 'rejuv_fac' : 1.0, 'rejuvflag' : 0, 'htpmb' : 1, 'ST_cr' : 1, 'ST_tide' : 1, 'bdecayfac' : 1, 'rembar_massloss' : 0.5, 'kickflag' : 0, 'zsun' : 0.014, 'bhms_coll_flag' : 0, 'don_lim' : -1, 'acc_lim' : -1}

bpp_noRe, bcm_noRe, initC_noRe, kick_info_noRe = Evolve.evolve(initialbinarytable=binary_set, BSEDict=BSEDict, timestep_conditions=timestep_conditions,
                                                                )


## YES Restart

In [9]:
%%capture
tl_restart = tidally_locked[(tidally_locked['t_quad'] < tidally_locked['t_remain']) ]


N,m1,m2,m3,porb,ecc,tphysf,tphys,kstar1,kstar2,kstar3,rad1,rad2,omega_spin1,omega_spin2,metallicity,tacc1,tacc2 = get_COSMIC_params(tl_restart)


binary_set = InitialBinaryTable.InitialBinaries(m1 = m1, m2 = m2, porb=porb, ecc=ecc, tphysf=tphysf,
                                                kstar1=kstar1, kstar2=kstar2, metallicity = metallicity, 
                                                rad_1=rad1, rad_2=rad2, ospin_1 = omega_spin1, ospin_2 = omega_spin2,
                                                tphys=tphys
                                                )

BSEDict = {'xi': 1.0, 'bhflag': 1, 'neta': 0.5, 'windflag': 3, 'wdflag': 1, 'alpha1': 1.0, 'pts1': 0.001, 'pts3': 0.02, 'pts2': 0.01, 'epsnov': 0.001, 'hewind': 0.5, 'ck': 1000, 'bwind': 0.0, 'lambdaf': 0.0, 'mxns': 3.0, 'beta': -1.0, 'tflag': 1, 'acc2': 1.5, 'grflag' : 1, 'remnantflag': 4, 'ceflag': 0, 'eddfac': 1.0, 'ifflag': 0, 'bconst': 3000, 'sigma': 265.0, 'gamma': -2.0, 'pisn': 45.0, 'natal_kick_array' : [[-100.0,-100.0,-100.0,-100.0,0.0], [-100.0,-100.0,-100.0,-100.0,0.0]], 'bhsigmafrac' : 1.0, 'polar_kick_angle' : 90, 'qcrit_array' : [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], 'cekickflag' : 2, 'cehestarflag' : 0, 'cemergeflag' : 0, 'ecsn' : 2.25, 'ecsn_mlow' : 1.6, 'aic' : 1, 'ussn' : 0, 'sigmadiv' :-20.0, 'qcflag' : 1, 'eddlimflag' : 0, 'fprimc_array' : [2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0], 'bhspinflag' : 0, 'bhspinmag' : 0.0, 'rejuv_fac' : 1.0, 'rejuvflag' : 0, 'htpmb' : 1, 'ST_cr' : 1, 'ST_tide' : 1, 'bdecayfac' : 1, 'rembar_massloss' : 0.5, 'kickflag' : 0, 'zsun' : 0.014, 'bhms_coll_flag' : 0, 'don_lim' : -1, 'acc_lim' : -1}

bpp_Re, bcm_Re, initC_Re, kick_info_Re = Evolve.evolve(
                                                        initialbinarytable=binary_set, BSEDict=BSEDict, timestep_conditions = timestep_conditions
                                                        )


## Evolve M3

In [10]:
def evolve_m3(which = 'Re'):
# Want to find the new type of m3 (and mass) during the time that the inner binary was evolving
    if which == 'Re' or which =='re':
        tl_df = tl_restart
    elif which == 'noRe' or which =='nore' or which=='no':
        tl_df = tl_no_restart


    N,m1,m2,m3,porb,ecc,tphysf,tphys,kstar1,kstar2,kstar3,rad1,rad2,omega_spin1,omega_spin2,metallicity,tacc1,tacc2 = get_COSMIC_params(tl_df)

    porb = [1e11] * len(tl_df) #make sure the outer is far out of the way
    #m1 = [0.05] * len(tl_df) # so no evolution
    m2 = m3 
    ecc = [0] * len(tl_df)
    kstar1 = [0] * len(tl_df)
    kstar2 = kstar3
    #tphys = [t*1e6 for t in tphys] #they were already in Myr, so function divided too many times, bring them back to Myr

    binary_set = InitialBinaryTable.InitialBinaries(m1 = m1, m2 = m3, porb=porb, ecc=ecc, tphysf=tphysf,
                                                    kstar1=kstar1, kstar2=kstar2, metallicity = metallicity, 
                                                    tphys=tphys
                                                    )

    timestep_conditions = [['binstate==0', 'dtp=200.0'], ['binstate==1', 'dtp=50.0']  ]

    BSEDict = {'xi': 1.0, 'bhflag': 1, 'neta': 0.5, 'windflag': 3, 'wdflag': 1, 'alpha1': 1.0, 'pts1': 0.001, 'pts3': 0.02, 'pts2': 0.01, 'epsnov': 0.001, 'hewind': 0.5, 'ck': 1000, 'bwind': 0.0, 'lambdaf': 0.0, 'mxns': 3.0, 'beta': -1.0, 'tflag': 1, 'acc2': 1.5, 'grflag' : 1, 'remnantflag': 4, 'ceflag': 0, 'eddfac': 1.0, 'ifflag': 0, 'bconst': 3000, 'sigma': 265.0, 'gamma': -2.0, 'pisn': 45.0, 'natal_kick_array' : [[-100.0,-100.0,-100.0,-100.0,0.0], [-100.0,-100.0,-100.0,-100.0,0.0]], 'bhsigmafrac' : 1.0, 'polar_kick_angle' : 90, 'qcrit_array' : [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], 'cekickflag' : 2, 'cehestarflag' : 0, 'cemergeflag' : 0, 'ecsn' : 2.25, 'ecsn_mlow' : 1.6, 'aic' : 1, 'ussn' : 0, 'sigmadiv' :-20.0, 'qcflag' : 1, 'eddlimflag' : 0, 'fprimc_array' : [2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0], 'bhspinflag' : 0, 'bhspinmag' : 0.0, 'rejuv_fac' : 1.0, 'rejuvflag' : 0, 'htpmb' : 1, 'ST_cr' : 1, 'ST_tide' : 1, 'bdecayfac' : 1, 'rembar_massloss' : 0.5, 'kickflag' : 0, 'zsun' : 0.014, 'bhms_coll_flag' : 0, 'don_lim' : -1, 'acc_lim' : -1}

    bpp_m3, bcm_m3, initC_m3, kick_info_m3 = Evolve.evolve(initialbinarytable=binary_set, 
                                                        BSEDict=BSEDict, timestep_conditions = [['kstar_1<16', 'dtp=50.0']])
    
    return (bpp_m3, bcm_m3, initC_m3, kick_info_m3)


In [11]:
(bpp_m3_noRe, bcm_m3_noRe, initC_m3_noRe, kick_info_m3_noRe) = evolve_m3(which = 'noRe')
(bpp_m3_Re, bcm_m3_Re, initC_m3_Re, kick_info_m3_Re)         = evolve_m3(which = 'Re')

## Insert "N" Column

In [12]:

def get_this_N(row): #row of initC dataframe
    this_final_row = tidally_locked[(  
                                (np.round(tidally_locked['e1'],4)      == np.round(row['ecc'],4))    & 
                                (np.round(tidally_locked['m1'],4)      == np.round(row['mass_1'],4)) &  
                                (np.round(tidally_locked['m2'],4)      == np.round(row['mass_2'],4)) 
                                #(np.round(tidally_locked['spintot'],3) == np.round(row['omega_spin_1'],3))
                            )]
    try:
        this_final_row['N'].tolist()[0]
    except:
        print(row.to_frame().T)
    return this_final_row['N'].tolist()[0]

    
def get_N_from_initC(row,initC):
    bin_num = int(row['bin_num'])
    this_initC_row = initC[initC['bin_num']==bin_num]
    return this_initC_row['N'].tolist()[0]
    
initC_Re['N']      = initC_Re.apply(lambda row: get_this_N(row), axis=1)
initC_noRe['N']    = initC_noRe.apply(lambda row: get_this_N(row), axis=1)
initC_m3_Re['N']   = initC_m3_Re.apply(lambda row: get_N_from_initC(row, initC = initC_Re), axis=1)
initC_m3_noRe['N'] = initC_m3_noRe.apply(lambda row: get_N_from_initC(row, initC = initC_noRe), axis=1)

initC_Re      = initC_Re[ ['N'] + [ col for col in initC_Re.columns if col != 'N' ] ] #move N columns to front
initC_noRe    = initC_noRe[ ['N'] + [ col for col in initC_noRe.columns if col != 'N' ] ] #move N columns to front
initC_m3_Re   = initC_m3_Re[ ['N'] + [ col for col in initC_m3_Re.columns if col != 'N' ] ] #move N columns to front
initC_m3_noRe = initC_m3_noRe[ ['N'] + [ col for col in initC_m3_noRe.columns if col != 'N' ] ] #move N columns to front
    
#add to other dfs
bpp_noRe['N']    = bpp_noRe.apply(lambda row: get_N_from_initC(row, initC = initC_noRe), axis=1)
bpp_Re['N']      = bpp_Re  .apply(lambda row: get_N_from_initC(row, initC = initC_Re), axis=1)
bpp_m3_Re['N']   = bpp_m3_Re.apply(lambda row: get_N_from_initC(row, initC = initC_Re), axis=1)
bpp_m3_noRe['N'] = bpp_m3_noRe.apply(lambda row: get_N_from_initC(row, initC = initC_noRe), axis=1)

bcm_Re['N']      = bcm_Re  .apply(lambda row: get_N_from_initC(row, initC = initC_Re), axis=1)
bcm_noRe['N']    = bcm_noRe.apply(lambda row: get_N_from_initC(row, initC = initC_noRe), axis=1)
bcm_m3_Re['N']   = bcm_m3_Re.apply(lambda row: get_N_from_initC(row, initC = initC_Re), axis=1)
bcm_m3_noRe['N'] = bcm_m3_noRe.apply(lambda row: get_N_from_initC(row, initC = initC_noRe), axis=1)

bcm_Re      = bcm_Re  [ ['N'] + [ col for col in bcm_Re.columns if col != 'N' ] ] #move N columns to front
bpp_Re      = bpp_Re  [ ['N'] + [ col for col in bpp_Re.columns if col != 'N' ] ] #move N columns to front
bcm_noRe    = bcm_noRe[ ['N'] + [ col for col in bcm_noRe.columns if col != 'N' ] ] #move N columns to front
bpp_noRe    = bpp_noRe[ ['N'] + [ col for col in bpp_noRe.columns if col != 'N' ] ] #move N columns to front
bpp_m3_Re   = bpp_m3_Re[ ['N'] + [ col for col in bpp_m3_Re.columns if col != 'N' ] ] #move N columns to front
bpp_m3_noRe = bpp_m3_noRe[ ['N'] + [ col for col in bpp_m3_noRe.columns if col != 'N' ] ] #move N columns to front
bcm_m3_Re   = bcm_m3_Re[ ['N'] + [ col for col in bcm_m3_Re.columns if col != 'N' ] ] #move N columns to front
bcm_m3_noRe = bcm_m3_noRe[ ['N'] + [ col for col in bcm_m3_noRe.columns if col != 'N' ] ] #move N columns to front


In [13]:
both_initC  = pd.concat([initC_noRe,initC_Re])
both_bpp    = pd.concat([bpp_noRe,bpp_Re])
both_bcm    = pd.concat([bcm_noRe,bcm_Re])
both_bpp_m3 = pd.concat([bpp_m3_noRe,bpp_m3_Re])
both_bcm_m3 = pd.concat([bcm_m3_noRe,bcm_m3_Re])

# Assign Evolution Type 

In [14]:
def insert_cols_cosmic(df,this_N=0):
    #df.insert(loc = 0, column = 'N', value = this_N)
    df.insert(loc = 1, column = 'Ad_or_Kick', value = 'Ad')
    df.insert(loc = 1, column = 'stop_type', value = 'time')
    df.insert(loc = 2, column = 'Restart', value = 'No')
    df.insert(loc = 3, column = 'a2_new', value = np.nan)
    df.insert(loc = 4, column = 'e2_new', value = np.nan)
    df.insert(loc = 5, column = 'i_new', value = np.nan)


## NO Restart                                ($t_{quad}$ > $t_{remain}$)

In [15]:
#columns that I will use for both bpp and bcm
common_columns = sorted( list( set(bpp_Re.columns.tolist()).intersection(bcm_Re.columns.tolist()) ) )

#keep only these common columns
bpp_noRe    = bpp_noRe[common_columns]
bcm_noRe    = bcm_noRe[common_columns]
bpp_Re      = bpp_Re[common_columns]
bcm_Re      = bcm_Re[common_columns]

bpp_m3_noRe = bpp_m3_noRe[common_columns]
bcm_m3_noRe = bcm_m3_noRe[common_columns]
bpp_m3_Re   = bpp_m3_Re[common_columns]
bcm_m3_Re   = bcm_m3_Re[common_columns]


In [16]:
#%%capture
## --------- t_quad > t_remain (NO Restart) ---------
cosmic_f_noRe = pd.DataFrame(columns=['Ad_or_Kick','stop_type','Restart','a2_new', 'e2_new','i_new','type3_new',
                                    'm3_new'] + common_columns )
#These binaries for sure don't need restart, so not too hard 
for i in bpp_noRe.index.unique():

    #initialize
    use_bcm = False
    t_Merge = np.nan
    t_WD = np.nan

    #bpp and bcm of this binary
    this_bpp = bpp_noRe.loc[i].reset_index(drop=True) 

    
    if len(this_bpp)==34: #if it is only one row long (nothing interesting happenned)
        this_bpp = bpp_noRe.loc[i].to_frame().T.reset_index(drop=True)
        use_bcm = True
        
    N       = this_bpp.loc[0]['N']
    bin_num = this_bpp.loc[0]['bin_num']

    if use_bcm: 
        this_bpp  = bcm_noRe[bcm_noRe['N']==N].reset_index(drop=True)
        if len(this_bpp)==34:
            this_bpp = bpp_noRe.loc[i].to_frame().T.reset_index(drop=True)
            
    this_bcm = bcm_noRe[bcm_noRe['N']==N].reset_index(drop=True)
    this_initC = initC_noRe.loc[bin_num] 
    start_time = this_initC['tphys']
    
    tl_row = tl_no_restart[tl_no_restart['N']==N] #final OSPE row
    
    if len(tl_row)==0: 
        print('no final OSPE row')
        continue
            
    insert_cols_cosmic(this_bpp,this_N = N)
    
    first_bpp_row = this_bpp.head(1)
    last_bpp_row  = this_bpp.tail(1)
    
    first_bcm_row = this_bcm.head(1)
    last_bcm_row  = this_bcm.tail(1)
    
    P2 = tl_row['P_out'].tolist()[0] / 1e6 # outer period in Myr

    t_ML = last_bpp_row.tphys.tolist()[0] #default to last time (will change if anything intersting happenned before then)
    duration_of_ML = t_ML - start_time #how long it evolved for
    
    interesting_evol_types = list(range(2,9))   #evol types of mass transfer or type change
    
    Merge_condition =  (this_bpp.sep <= 0.)    #this_bpp.evol_type.isin(interesting_evol_types)  #Condiiton for interesting event
    WD_condition    =  ( (this_bpp.kstar_1.isin(WD_Types)) | (this_bpp.kstar_2.isin(WD_Types)) )
                    
    t_Merge = this_bpp[Merge_condition].tphys.min()  #time of Mass Loss (time until interesting evvent)
    t_WD    = this_bpp[WD_condition].tphys.min()
    
    
    if not np.isnan(t_Merge):  #if merged, then this is it 
        t_ML = t_Merge
        duration_of_ML = t_Merge - start_time
    if not np.isnan(t_WD):
        duration_of_ML = t_WD - start_time
        
    if (this_bpp.sep == 0.) .any():  #USE KICK if there are any mergers or simulated time<P2/2, Kick Prescription

        this_bpp=this_bpp.reset_index(drop=True)
        row_merged = this_bpp[(this_bpp.kstar_1 == 15.) | (this_bpp.kstar_2 == 15.)]#.reset_index(drop=True)
        row_merged_idx = row_merged.index.values.tolist()

        if len(this_bpp)>=3:
            row_before_merger = this_bpp[(this_bpp['kstar_1'] !=15.) & (this_bpp['kstar_2'] !=15.)][-1:]
            #row_before_merger = this_bpp.iloc[[   row_merged_idx[0] - 1   ]] #get row before merge happenned 
            row_of_merger =     this_bpp.iloc[[   row_merged_idx[0]       ]] #redudnat, but looks nice
            #row_after_merger =  this_bpp.iloc[[   row_merged_idx[0] + 1   ]]
            row_after_merger =  this_bpp[-1:]

        elif len(this_bpp)==2:
            row_before_merger, row_of_merger,row_after_merger = this_bpp.loc[0].to_frame().T, \
                                                                this_bpp.loc[1].to_frame().T, \
                                                                this_bpp.loc[1].to_frame().T
                                                                
        
        row_before_merger['stop_type'] = 'merged'
        row_of_merger['stop_type']     = 'merged'
        row_after_merger['stop_type']  = 'merged'
                   
        t_ML = row_of_merger.tphys.tolist()[0]
        if duration_of_ML < P2:
             row_before_merger['Ad_or_Kick'] = 'Kick'
             row_of_merger['Ad_or_Kick']     = 'Kick'
             row_after_merger['Ad_or_Kick']  = 'Kick'
             
             
        cosmic_f_noRe = cosmic_f_noRe.append(row_before_merger, ignore_index=True)
        cosmic_f_noRe = cosmic_f_noRe.append(row_of_merger    , ignore_index=True)
        cosmic_f_noRe = cosmic_f_noRe.append(row_after_merger,  ignore_index=True)
        
    #Update a1 and a2 in OSPE output file
    else: #If they did not merge
        if duration_of_ML < P2:
            this_bpp['Ad_or_Kick'] = 'Kick'
        
        cosmic_f_noRe = cosmic_f_noRe.append(this_bpp.tail(3), ignore_index=True) #append last three rows
        
            

## YES Restart                                ($t_{quad}$ < $t_{remain}$)

In [17]:
#%%capture
## --------- t_quad > t_remain (NO Restart) ---------
changed_bpp_columns = list( set(this_bcm.columns.tolist()).intersection(this_bpp.columns.tolist()) )
cosmic_f_Re = pd.DataFrame(columns=['Ad_or_Kick','stop_type','Restart','a2_new', 'e2_new','i_new','type3_new',
                                    'm3_new'] + changed_bpp_columns )
#These binaries for sure don't need restart, so not too hard 
for i in bpp_Re.index.unique():

    #initialize
    use_bcm = False
    t_Merge = np.nan
    t_WD = np.nan
    
    #bpp and bcm of this binary
    this_bpp = bpp_Re.loc[i].reset_index(drop=True) 

    if len(this_bpp)==34: #  if it is only one row long (nothing interesting happenned)
        this_bpp = bpp_Re.loc[i].to_frame().T.reset_index(drop=True)
        use_bcm = False
        
    N       = this_bpp.loc[0]['N']
    bin_num = this_bpp.loc[0]['bin_num']
    
    this_bcm = bcm_Re[bcm_Re['N']==N].reset_index(drop=True)
    this_initC = initC_Re.loc[bin_num] 
    tl_row = tl_restart[tl_restart['N']==N] #final OSPE row
    
    start_time = this_initC['tphys']
    t_quad = tl_row['t_quad'].tolist()[0] / 1e6

    if use_bcm: 
        if len(this_bcm) == 34 and type(this_bcm) != pd.core.frame.DataFrame:
            this_bcm = this_bcm.to_frame().T
            this_bcm.columns = bcm_Re.columns
        this_bpp = this_bcm #use bcm instead
    
    
    if len(tl_row)==0: 
        print('no final OSPE row')
        continue
            
    insert_cols_cosmic(this_bpp,this_N = N)
    insert_cols_cosmic(this_bcm,this_N = N)

    first_bpp_row = this_bpp.head(1)
    last_bpp_row  = this_bpp.tail(1)
    
    first_bcm_row = this_bcm.head(1)
    last_bcm_row  = this_bcm.tail(1)
    
    P2 = tl_row['P_out'].tolist()[0] / 1e6 # outer period in Myr
    t_end = last_bpp_row.tphys.tolist()[0]
    
    t_ML = t_end  #default to last time (will change if anything intersting happenned before then)
    
    duration_of_ML = t_ML - start_time #how long it evolved for
    
    interesting_evol_types = [2,4,6,8,9]
    
    Merge_condition        = (this_bpp.sep <= 0.)    #this_bpp.evol_type.isin(interesting_evol_types)  #Condiiton for interesting event
    WD_condition           = ( (this_bpp.kstar_1.isin(WD_Types)) | (this_bpp.kstar_2.isin(WD_Types)) )#& ((this_bpp.RRLO_1 < 1) & (this_bpp.RRLO_2 < 1)))     
    WD_condition_noRRLO    = ( WD_condition & ((this_bpp.RRLO_1 < 1) & (this_bpp.RRLO_2 < 1)))     
    # ML_condition           = ( this_bpp.evol_type.isin(interesting_evol_types) )#& ((this_bpp.RRLO_1 < 1.) & (this_bpp.RRLO_2 < 1.)))   #this_bpp.evol_type.isin(interesting_evol_types)  #Condiiton for interesting event
    # ML_condition_noRRLO    = ( ML_condition & ((this_bpp.RRLO_1 < 1.) & (this_bpp.RRLO_2 < 1.)))   #this_bpp.evol_type.isin(interesting_evol_types)  #Condiiton for interesting event

    t_Merge           = this_bpp[Merge_condition]       .tphys.min()  #time of Mass Loss (time until interesting evvent)
    t_WD              = this_bpp[WD_condition]          .tphys.min()  #time of Mass Loss (time until interesting evvent)
    t_WD_noRRLO       = this_bpp[WD_condition_noRRLO]   .tphys.min()  #time of Mass Loss (time until interesting evvent)
    # t_Transfer        = this_bpp[ML_condition]          .tphys.max()  #time of Mass Loss (time until interesting evvent)
    # t_Transfer_noRRLO = this_bpp[ML_condition_noRRLO]   .tphys.min()  #time of Mass Loss (time until interesting evvent)

    if not np.isnan(t_Merge):  #if merged, then this is it 
        t_ML = t_Merge
        duration_of_ML = t_Merge - start_time
        
    if not np.isnan(t_WD):
        duration_of_ML = t_WD - start_time
        
    end_ML_row = this_bpp.loc[ (this_bpp['RRLO_1'] >1) & (this_bpp['RRLO_2'] >1) ][-1:] #get last line or RLO, then this is the true t_ML
    if len(end_ML_row) !=0:
        t_ML = end_ML_row.tphys.tolist()[0]
        duration_of_ML = t_ML - start_time
    elif not np.isnan(t_WD):  #else go for the first time that it became a WD       
        t_ML = t_WD   # when one star becomes WD
        duration_of_ML = t_WD - start_time
    elif not np.isnan(t_Merge): #else go for the first time that it merged
        t_ML = t_Merge  # when one stars merge
        duration_of_ML = t_Merge - start_time
    #if they are merged in last line, cant put back into OSPE   
    if (this_bpp[-1:].RRLO_1.tolist()[0] >=1) or (this_bpp[-1:].RRLO_2.tolist()[0]>=1): 
        t_ML=t_end
        duration_of_ML = t_end - start_time
        
    if (this_bpp.sep == 0.) .any():  #USE KICK if there are any mergers or simulated time<P2/2, Kick Prescription

        this_bpp=this_bpp.reset_index(drop=True)
        row_merged = this_bpp[(this_bpp.kstar_1 == 15.) | (this_bpp.kstar_2 == 15.)]#.reset_index(drop=True)
        row_merged_idx = row_merged.index.values.tolist()

        if len(this_bpp)>=3:
            row_before_merger = this_bpp[(this_bpp['kstar_1'] !=15.) & (this_bpp['kstar_2'] !=15.)][-1:]
            #row_before_merger = this_bpp.iloc[[   row_merged_idx[0] - 1   ]] #get row before merge happenned 
            row_of_merger =     this_bpp.iloc[[   row_merged_idx[0]       ]] #redudnat, but looks nice
            #row_after_merger =  this_bpp.iloc[[   row_merged_idx[0] + 1   ]]
            row_after_merger =  this_bpp[-1:]

        elif len(this_bpp)==2:
            row_before_merger, row_of_merger,row_after_merger = this_bpp.loc[0].to_frame().T, \
                                                                this_bpp.loc[1].to_frame().T, \
                                                                this_bpp.loc[1].to_frame().T
                                                                


        row_before_merger['stop_type'] = 'merged'
        row_of_merger['stop_type']     = 'merged'
        row_after_merger['stop_type']  = 'merged'
                   
        t_ML = row_of_merger.tphys.tolist()[0]
        
        if row_before_merger.tphys.tolist()[0] == 0. : #it was already a merger, so don't do anything to update params
            row_before_merger['Ad_or_Kick'] = 'alr_merged'
            row_of_merger    ['Ad_or_Kick'] = 'alr_merged'
            row_after_merger ['Ad_or_Kick'] = 'alr_merged'
            
        elif duration_of_ML < P2:
             row_before_merger['Ad_or_Kick'] = 'Kick'
             row_of_merger['Ad_or_Kick']     = 'Kick'
             row_after_merger['Ad_or_Kick']  = 'Kick'
             
             
        cosmic_f_Re = cosmic_f_Re.append(row_before_merger, ignore_index=True)
        cosmic_f_Re = cosmic_f_Re.append(row_of_merger    , ignore_index=True)
        cosmic_f_Re = cosmic_f_Re.append(row_after_merger,  ignore_index=True)
        

    #If DID NOT MERGE
    else: 
        
        if t_ML != t_end: #something interesting occurred before end time
            bpp_after_tquad = this_bpp[this_bpp['tphys']>=(start_time + t_quad)]
            bcm_after_tquad = this_bcm[this_bcm['tphys']>=(start_time + t_quad)]

            if len(bpp_after_tquad) == 0:
                bcm_after_tquad = this_bcm

            row_of_interest = None
            # if one of them is a WD, no RRLO, no tidally locked
            row_of_interest = bcm_after_tquad[ ( (bcm_after_tquad.kstar_1.isin(WD_Types)) | (bcm_after_tquad.kstar_2.isin(WD_Types)) ) & 
                                            ((bcm_after_tquad.RRLO_1 < 0.5) & (bcm_after_tquad.RRLO_2 < 0.5)) & 
                                            ((bcm_after_tquad.sep > 25) | (bcm_after_tquad.ecc > 0.005)) ]
            #first row without RLO and tidally locked
            if len(row_of_interest) == 0:
                row_of_interest = bcm_after_tquad[ ((bcm_after_tquad.RRLO_1 < 0.5) & (bcm_after_tquad.RRLO_2 < 0.5)) & 
                                                ((bcm_after_tquad.sep > 25) | (bcm_after_tquad.ecc > 0.005)) ]
            #second to last row then 
            if len(row_of_interest) == 0:
                row_of_interest = bcm_after_tquad[-2:-1] #second to last row
                
            row_of_interest_idx = row_of_interest.index.values.tolist()
            idx_of_interest = row_of_interest_idx[0]
            
            if idx_of_interest == 0:
                idx_of_interest += 1
            #print(N,this_df.bin_num.tolist()[0])
            
            
            row_before = this_bcm.loc [[   idx_of_interest - 1   ]] #get row before merge happenned 
            row_of     = this_bcm.loc [[   idx_of_interest       ]] #redudnat, but looks nice
            row_after  = this_bcm.loc [[   idx_of_interest + 1   ]]
            
            #upate columns accordingly
            this_bpp['stop_type'] = 'interesting'
            
            row_before['stop_type'] = 'interesting'
            row_of    ['stop_type'] = 'interesting'
            row_after ['stop_type'] = 'interesting'
            
            if len(bpp_after_tquad) == 0:
                row_before['Restart'] = 'No'
                row_of    ['Restart'] = 'No'
                row_after ['Restart'] = 'No'
            else:
                row_before['Restart'] = 'Yes'
                row_of    ['Restart'] = 'Yes'
                row_after ['Restart'] = 'Yes'
            
            if duration_of_ML < P2:
                row_before['Ad_or_Kick'] = 'Kick'
                row_of    ['Ad_or_Kick'] = 'Kick'
                row_after ['Ad_or_Kick'] = 'Kick'
            
            cosmic_f_Re = cosmic_f_Re.append(row_before, ignore_index=True)
            cosmic_f_Re = cosmic_f_Re.append(row_of,     ignore_index=True)
            cosmic_f_Re = cosmic_f_Re.append(row_after,  ignore_index=True)

        elif t_ML == t_end:
            cosmic_f_Re = cosmic_f_Re.append(this_bpp.tail(3), ignore_index=True) #append last three rows
        
        """if t_ML != t_end: #if something interesting happenned 
            
            row_of_interest = this_df[ this_df.tphys == t_ML   ][-1:] #row of interesting event
            #if not np.isnan(t_WD): print(rows_of_interst)
            row_of_interest_idx = row_of_interest.index.values.tolist()
            #print(N,this_df.bin_num.tolist()[0])
            row_before = this_df.iloc [[   row_of_interest_idx[0] - 1   ]] #get row before merge happenned 
            row_of     = this_df.iloc [[   row_of_interest_idx[0]       ]] #redudnat, but looks nice
            row_after  = this_df.iloc [[   row_of_interest_idx[0] + 1   ]]
            
            this_df['stop_type'] = 'interesting'
            
            row_before['stop_type'] = 'interesting'
            row_of    ['stop_type'] = 'interesting'
            row_after ['stop_type'] = 'interesting'
            
            row_before['Restart'] = 'Yes'
            row_of    ['Restart'] = 'Yes'
            row_after ['Restart'] = 'Yes'
            
            if t_ML < P2:
                row_before['Ad_or_Kick'] = 'Kick'
                row_of    ['Ad_or_Kick'] = 'Kick'
                row_after ['Ad_or_Kick'] = 'Kick'
            
            cosmic_f_Re = cosmic_f_Re.append(row_before, ignore_index=True)
            cosmic_f_Re = cosmic_f_Re.append(row_of,     ignore_index=True)
            cosmic_f_Re = cosmic_f_Re.append(row_after,  ignore_index=True)
            
        else: #if t_ML=t_end, then no no need for restart 
            
            if t_ML < P2:
                this_df['Ad_or_Kick'] = 'Kick'
            
            this_df['Restart'] = 'No'
            
            cosmic_f_Re = cosmic_f_Re.append(this_df.tail(3), ignore_index=True)
            """
            

In [18]:

"""  
For non-mergers, want to find the last line in bpp to which I can restart file
1. First go to the last line after which a t_quad has passed (recall, all of these have a t_quad length before they end)
        would look something like bpp_Re[bpp_Re['tphys'] >= start_time + t_quad] --> Do the same for bcm_Re
1. Find the first time after that to which no RLO, so RRLO_1 and RRLO_2 < 0.8 (only need to check bpp I think)
2. Make sure that they are not tidally locked, no point in putting back in OSPE if they are
3. If there was no intersting change during the end time, i.e. the length of bcm_Re is 2, then def doesnt need Restart

Questions:
If the binary ever coalesced, should I even cosndier putting them back into OSPE?
"""


"  \nFor non-mergers, want to find the last line in bpp to which I can restart file\n1. First go to the last line after which a t_quad has passed (recall, all of these have a t_quad length before they end)\n        would look something like bpp_Re[bpp_Re['tphys'] >= start_time + t_quad] --> Do the same for bcm_Re\n1. Find the first time after that to which no RLO, so RRLO_1 and RRLO_2 < 0.8 (only need to check bpp I think)\n2. Make sure that they are not tidally locked, no point in putting back in OSPE if they are\n3. If there was no intersting change during the end time, i.e. the length of bcm_Re is 2, then def doesnt need Restart\n\nQuestions:\nIf the binary ever coalesced, should I even cosndier putting them back into OSPE?\n"

In [19]:
# %%capture
""" *OLD*
#%%capture
## --------- t_quad > t_remain (NO Restart) ---------
cosmic_f_Re = pd.DataFrame(columns=['Ad_or_Kick','stop_type','Restart','a2_new', 'e2_new','i_new','type3_new',
                                    'm3_new'] + bpp_noRe.columns.tolist() )


for i in bpp_Re.index.unique():
    this_df  = bpp_Re.loc[i].reset_index(drop=True) # relevant columns for this binary
    
    if len(this_df)==45: 
        try:
            this_df = bpp_Re.loc[i].to_frame().T.reset_index(drop=True)
        except:
            this_df = this_df.to_frame().T.reset_index(drop=True)
        
        this_df.columns = bpp_noRe.columns
            
        #continue #if there is only one row

    N = this_df.loc[0].N
    bin_num = this_df.loc[0].bin_num
    this_initC = initC_Re.loc[bin_num]
    this_bcm = bcm_Re.loc[i].reset_index(drop=True) # relevant columns for this binary
    tl_row = tl_restart[tl_restart['N']==N]
    if len(tl_row)==0: continue
    if len(this_bcm) == 40 and type(this_bcm) != pd.core.frame.DataFrame:
        this_bcm = this_bcm.to_frame().T
        this_bcm.columns = bcm_noRe.columns


    insert_cols_cosmic(this_df,this_N = N)
    insert_cols_cosmic(this_bcm,this_N = N)    
    
    last_bpp_row = this_df[-1:]
    t_end = last_bpp_row.tphys.tolist()[0] 
    
    t_ML = t_end  #default to last time
    P2 = tl_row.P_out.tolist()[0] /1e6 #in Myr

    interesting_evol_types = list(range(2,9))   #evol types of mass transfer or type change
    interesting_evol_types = [2,4,6,8,9]
    
    WD_condition    =  ( (this_df.kstar_1.isin(WD_Types)) | (this_df.kstar_2.isin(WD_Types)) & ((this_df.RRLO_1 < 1) & (this_df.RRLO_2 < 1)))     
    Merge_condition =  ( this_df.sep == 0. )              #this_df.evol_type.isin(interesting_evol_types)  #Condiiton for interesting event
    ML_condition    =  ( this_df.evol_type.isin(interesting_evol_types) & ((this_df.RRLO_1 < 1.) & (this_df.RRLO_2 < 1.)))   #this_df.evol_type.isin(interesting_evol_types)  #Condiiton for interesting event
   
    t_WD =       this_df[WD_condition]   .tphys.min()  #time of Mass Loss (time until interesting evvent)
    t_Merge =    this_df[Merge_condition].tphys.min()  #time of Mass Loss (time until interesting evvent)
    t_Transfer = this_df[ML_condition]   .tphys.max()  #time of Mass Loss (time until interesting evvent)

         
    end_ML_row = this_df.loc[ (this_df['RRLO_1'] >1) & (this_df['RRLO_2'] >1) ][-1:] #if still roche at end time
    if len(end_ML_row) !=0:
        t_ML = end_ML_row.tphys.tolist()[0]
    elif not np.isnan(t_WD):           
        t_ML = t_WD   # when one star becomes WD
    elif not np.isnan(t_Merge):      
        t_ML = t_Merge  # when one stars merge
    if (this_df[-1:].RRLO_1.tolist()[0] >=1) or (this_df[-1:].RRLO_2.tolist()[0]>=1): #if they are merged in last line, cant put back into OSPE
        t_ML=t_end
    

    #this_df.columns = ['N'] + this_df.columns.tolist()
    if (this_df.porb == 0.).any():  #if there are any mergers 

        rows_merged = this_df[(this_df.kstar_1 == 15.) | (this_df.kstar_2 == 15.)]
        row_merged_idx = rows_merged.index.values.tolist()
        
        if len(this_df)>=3:
            row_before_merger = this_df[(this_df['kstar_1'] !=15.) & (this_df['kstar_2'] !=15.)][-1:]
            #row_before_merger = this_df.iloc[[   row_merged_idx[0] - 1   ]] #get row before merge happenned 
            row_of_merger =     this_df.iloc[[   row_merged_idx[0]       ]] #redudnat, but looks nice
            #row_after_merger =  this_df.iloc[[   row_merged_idx[0] + 1   ]]
            row_after_merger =  this_df[-1:]
        elif len(this_df)==2:
            row_before_merger, row_of_merger,row_after_merger = this_df.loc[0].to_frame().T, \
                                                                this_df.loc[1].to_frame().T, \
                                                                this_df.loc[1].to_frame().T
            row_before_merger = this_df[(this_df['kstar_1'] !=15.) & (this_df['kstar_2'] !=15.)][-1:]
            
        row_before_merger['stop_type'] = 'merged'
        row_of_merger['stop_type']     = 'merged'
        row_after_merger['stop_type']  = 'merged'
        
        row_before_merger['Restart'] = 'No'
        row_of_merger['Restart']     = 'No'
        row_after_merger['Restart']  = 'No'
        
        if row_before_merger.tphys.tolist()[0] == 0. : #it was already a merger, so don't do anything
            row_before_merger['Ad_or_Kick'] = 'alr_merged'
            row_of_merger    ['Ad_or_Kick'] = 'alr_merged'
            row_after_merger ['Ad_or_Kick'] = 'alr_merged'
             
        elif t_ML < P2:
            row_before_merger['Ad_or_Kick'] = 'Kick'
            row_of_merger    ['Ad_or_Kick'] = 'Kick'
            row_after_merger ['Ad_or_Kick'] = 'Kick'
             
             
        cosmic_f_Re = cosmic_f_Re.append(row_before_merger, ignore_index=True)
        cosmic_f_Re = cosmic_f_Re.append(row_of_merger,     ignore_index=True)
        cosmic_f_Re = cosmic_f_Re.append(row_after_merger,  ignore_index=True)
            
    
    else: #If they did not merge, APPEND the three rows about the line of the interesting event (line under, the event ine, and line after (if there is one))
        
        if t_ML != t_end: #if something interesting happenned 
            
            row_of_interest = this_df[ this_df.tphys == t_ML   ][-1:] #row of interesting event
            #if not np.isnan(t_WD): print(rows_of_interst)
            row_of_interest_idx = row_of_interest.index.values.tolist()
            #print(N,this_df.bin_num.tolist()[0])
            row_before = this_df.iloc [[   row_of_interest_idx[0] - 1   ]] #get row before merge happenned 
            row_of     = this_df.iloc [[   row_of_interest_idx[0]       ]] #redudnat, but looks nice
            row_after  = this_df.iloc [[   row_of_interest_idx[0] + 1   ]]
            
            this_df['stop_type'] = 'interesting'
            
            row_before['stop_type'] = 'interesting'
            row_of    ['stop_type'] = 'interesting'
            row_after ['stop_type'] = 'interesting'
            
            row_before['Restart'] = 'Yes'
            row_of    ['Restart'] = 'Yes'
            row_after ['Restart'] = 'Yes'
            
            if t_ML < P2:
                row_before['Ad_or_Kick'] = 'Kick'
                row_of    ['Ad_or_Kick'] = 'Kick'
                row_after ['Ad_or_Kick'] = 'Kick'
            
            cosmic_f_Re = cosmic_f_Re.append(row_before, ignore_index=True)
            cosmic_f_Re = cosmic_f_Re.append(row_of,     ignore_index=True)
            cosmic_f_Re = cosmic_f_Re.append(row_after,  ignore_index=True)
            
        else: #if t_ML=t_end, then no no need for restart 
            
            if t_ML < P2:
                this_df['Ad_or_Kick'] = 'Kick'
            
            this_df['Restart'] = 'No'
            
            cosmic_f_Re = cosmic_f_Re.append(this_df.tail(3), ignore_index=True)
        # if N == 6261:
        #     df = this_df
            
"""

" *OLD*\n#%%capture\n## --------- t_quad > t_remain (NO Restart) ---------\ncosmic_f_Re = pd.DataFrame(columns=['Ad_or_Kick','stop_type','Restart','a2_new', 'e2_new','i_new','type3_new',\n                                    'm3_new'] + bpp_noRe.columns.tolist() )\n\n\nfor i in bpp_Re.index.unique():\n    this_df  = bpp_Re.loc[i].reset_index(drop=True) # relevant columns for this binary\n    \n    if len(this_df)==45: \n        try:\n            this_df = bpp_Re.loc[i].to_frame().T.reset_index(drop=True)\n        except:\n            this_df = this_df.to_frame().T.reset_index(drop=True)\n        \n        this_df.columns = bpp_noRe.columns\n            \n        #continue #if there is only one row\n\n    N = this_df.loc[0].N\n    bin_num = this_df.loc[0].bin_num\n    this_initC = initC_Re.loc[bin_num]\n    this_bcm = bcm_Re.loc[i].reset_index(drop=True) # relevant columns for this binary\n    tl_row = tl_restart[tl_restart['N']==N]\n    if len(tl_row)==0: continue\n    if len(this_bcm) 

# Calculate New Parameters

In [20]:
def c(theta=0):
    theta=np.random.uniform(0,np.pi)
    return theta

In [21]:

## -------------- Function to Update Files --------------
def update_params(cosmic_df, tl_df, bpp_m3 = bpp_m3_noRe):
    
    for i, row in cosmic_df.iterrows():
        
        bin_num = row.bin_num

        tl_row = tl_df[tl_df.N == row.N]
        bpp_m3_row = bpp_m3[bpp_m3['bin_num'] == bin_num].reset_index(drop=True).iloc[-1:]
    
        N = row.N
        t = row.tphys 
        m1, m2, m3 = row.mass_1, row.mass_2, bpp_m3_row.mass_2.to_list()[0]
        R1, R2 = row.rad_1, row.rad_2
        e1, sep, p_orb = row.ecc, row.sep * 0.00465047, row.porb / 365.25 #sep in AU, p_orb in yr
        ospin1, ospin2 = row.omega_spin_1, row.omega_spin_2
        type1, type2, type3 = row.kstar_1, row.kstar_2, bpp_m3_row.kstar_2
        a1 = np.power( ( (p_orb**2)*(m1+m2) ) ,1./3.) #Kepler

        #old parameters (from OSPE output file)
        m1_old,m2_old,m3_old = tl_row.m1.to_list()[0], tl_row.m2.to_list()[0], tl_row.m3.to_list()[0]
        e1_old, e2_old = tl_row.e1.to_list()[0], tl_row.e2.to_list()[0]
        a1_old, a2_old = tl_row.a1.to_list()[0], tl_row.a2.to_list()[0]
        i_old, i1, i2 = np.deg2rad(tl_row.i.to_list()[0]), np.deg2rad(tl_row.i1.to_list()[0]), np.deg2rad(tl_row.i2.to_list()[0])
        R1_old, R2_old = tl_row.R1.to_list()[0]*215.032, tl_row.R2.to_list()[0]*215.032  #in R_sun
        ospin1_old, ospin2_old = tl_row.spintot.to_list()[0], np.sqrt(tl_row.spin2h**2.+tl_row.spin2e**2.+tl_row.spin2q**2.) .tolist()[0]
        
        
        #calculate new params using kick prescription
        if row['Ad_or_Kick'] == 'Kick':
            #If they are in common envelope, then approximate triple as binary with inner m1+m2
            if m1==0. and m2 == 0:
                new_a2,new_e2,new_i = np.nan,np.nan,np.nan
            else: #row.RRLO_1 >=1 or row.RRLO_2 >=1: , KICK

                EA1 = sample_angle()
                EA2 = sample_angle()
                
                R = get_r_mag(a2_old,e2_old,EA2) #outer separation before merger
                r = get_r_mag(a1,e1,EA1)         #inner separation before merger
                
                vr = get_vr(m1+m2, m3, R, a2_old)
                
                new_a2 = get_a_new(m1 = m3_old, m2 = m1_old+m2_old, m2n = m1+m2, 
                                     e1=e2_old, EA1=EA2, a_old = a2_old) #m2_new = m3,
                
                
                new_e2 = get_e1_new(R,vr,new_a2,m1+m2,m3)

                L1n, L2n = get_L1(m1,m2,a1),             get_L2(m1,m2,m3,new_a2)
                L1o, L2o = get_L1(m1_old,m2_old,a1_old), get_L2(m1_old,m2_old,m3_old,a2_old)
                
                h1n, h2n = get_h(m1,m2,a1,e1), get_h(m1+m2,m3,new_a2,new_e2) 
                h1o, h2o = get_h(m1_old,m2_old,a1_old,e1_old), get_h(m1_old+m2_old,m3_old,a2_old,e2_old) 
                
                G1n, G2n = get_G_from_h(m1,m2,h1n), get_G_from_h(m1+m2,m3,h2n)
                G1o, G2o = get_G_from_h(m1_old,m2_old,h1o), get_G_from_h(m1_old+m2_old,m3_old,h2o)
                   
                if m1 ==0.:
                    L1n = get_L_rigid(m2,R2,ospin2)
                    L1o = get_L_rigid(m2_old,R2_old,ospin2_old)
                    G1n, G2n = get_G(L1n,e1), get_G(L2n,new_e2)
                    G1o, G2o = get_G(L1o,e1_old), get_G(L2o,e2_old)
                elif m2 ==0.:
                    L1n = get_L_rigid(m1,R1,ospin1)  
                    L1o = get_L_rigid(m1_old,R1_old,ospin1_old)  
                    G1n, G2n = get_G(L1n,e1), get_G(L2n,new_e2)
                    G1o, G2o = get_G(L1o,e1_old), get_G(L2o,e2_old)
                    
                #G1n, G2n = get_G(L1n,e1), get_G(L2n,new_e2)
                #G1o, G2o = get_G(L1o,e1_old), get_G(L2o,e2_old)
                
                Gtot   = new_Gtot_from_old((m1_old,m2_old,m3),(m1,m2,m3),G1o,G2o,i_old,e1_old, e1 )             #G1n*np.cos(i1) + G2n*np.cos(i2)
                my_i_n = new_i_from_stuff ((m1_old,m2_old,m3_old),(m1,m2,m3),
                                           i_old*180/pi,e1_old,e1,e2_old,a1,new_a2)             #G1n*np.cos(i1) + G2n*np.cos(i2)

                new_i = get_i(G1n,G2n,Gtot) 
                cosi = (Gtot * Gtot - G1n*G1n - G2n*G2n ) / 2*G1n*G2n
                #my_i_n = np.arccos(cosi)
                #if new_i == np.nan: print(G1n,G2n,Gtot)

                cosmic_df.at[i,'type3_new'] = type3.tolist()[0]#.tolist()[0]  if type(type3) != float else type3 #.=[0]  if type(type3)  != float else type3
                cosmic_df.at[i,'m3_new']    = m3#.tolist()[0]     if type(m3) != float else m3    #.values[0]   if type(m3)     != float else m3
                cosmic_df.at[i,'a2_new']    = new_a2#.tolist()[0] if type(new_a2) != float else new_a2
                cosmic_df.at[i,'e2_new']    = new_e2#.tolist()[0] if type(new_e2) != float else new_e2 
                cosmic_df.at[i,'i_new']     = my_i_n#.tolist()[0] if type(my_i_n) != float else my_i_n #.values[0] if type(my_i_n) != float else my_i_n
        
        elif row['Ad_or_Kick'] == 'Ad':
            #calculate new params using adiabatic prescription
            # if row.RRLO_1 >=1 or row.RRLO_2 >=1: #If they are in common envelope, estimate as binary
            if m1==0. and m2 == 0.:
                new_a2,new_e2,new_i = np.nan,np.nan,np.nan
            else:
                
                new_a2 = get_a_adiabatic((m1_old,m2_old,m3_old),(m1,m2,m3),a2_old)
                new_e2 = e2_old 

                """
                L1n, L2n = get_L1(m1,m2,a1),             get_L2(m1,m2,m3,new_a2)
                L1o, L2o = get_L1(m1_old,m2_old,a1_old), get_L2(m1_old,m2_old,m3_old,a2_old)
                
                h1n, h2n = get_h(m1,m2,a1,e1), get_h(m1+m2,m3,new_a2,new_e2) 
                h1o, h2o = get_h(m1_old,m2_old,a1_old,e1_old), get_h(m1_old+m2_old,m3_old,a2_old,e2_old) 
                
                G1n, G2n = get_G_from_h(m1,m2,h1n), get_G_from_h(m1+m2,m3,h2n)
                G1o, G2o = get_G_from_h(m1_old,m2_old,h1o), get_G_from_h(m1_old+m2_old,m3_old,h2o)
                   
                if m1 ==0.:
                    L1n = get_L_rigid(m2,R2,ospin2)
                    L1o = get_L_rigid(m2_old,R2_old,ospin2_old)
                    G1n, G2n = get_G(L1n,e1), get_G(L2n,new_e2)
                    G1o, G2o = get_G(L1o,e1_old), get_G(L2o,e2_old)
                elif m2 ==0.:
                    L1n = get_L_rigid(m1,R1,ospin1)  
                    L1o = get_L_rigid(m1_old,R1_old,ospin1_old)  
                    G1n, G2n = get_G(L1n,e1), get_G(L2n,new_e2)
                    G1o, G2o = get_G(L1o,e1_old), get_G(L2o,e2_old)
                
                Gtot  = new_Gtot_from_old((m1_old,m2_old,m3),(m1,m2,m3),G1o,G2o,i_old,e1_old,e1 )        
                Gtoto = G1o*np.cos(i1) + G2o*np.cos(i2)       
                #Gtot =G1n*np.cos(i1) + G2n*np.cos(i2)
                #print(Gtot)

                cosi = (Gtot**2 - G1n**2 - G2n**2 ) / (2*G1n*G2n)
                cosio = (Gtoto**2 - G1o**2 - G2o**2 ) / (2*G1o*G2o)
                new_i = get_i(G1n,G2n,Gtot)
                """
                my_i_n = new_i_from_stuff ((m1_old,m2_old,m3_old),(m1,m2,m3),
                                              i_old*180/pi,e1_old,e1,e2_old, a1, new_a2 )             #G1n*np.cos(i1) + G2n*np.cos(i2)

                #new_omega = np.rad2deg(sample_angle())
            
            cosmic_df.at[i,'type3_new'] = type3.tolist()[0]  #.tolist()[0] if type(type3) != float else type3 #.=[0]  if type(type3)  != float else type3
            cosmic_df.at[i,'m3_new']    = m3  #tolist()[0] if type(m3) != float else m3    #.values[0]   if type(m3)     != float else m3
            cosmic_df.at[i,'a2_new']    = new_a2  #.tolist()[0] if type(new_a2) != float else new_a2
            cosmic_df.at[i,'e2_new']    = new_e2  #.tolist()[0] if type(new_e2) != float else new_e2 
            cosmic_df.at[i,'i_new']     = my_i_n  #.tolist()[0] if type(my_i_n) != float else my_i_n #.values[0] if type(my_i_n) != float else my_i_n


In [22]:
%%capture
update_params(cosmic_f_noRe, tl_no_restart, bpp_m3 = bcm_m3_noRe)

In [23]:
%%capture
update_params(cosmic_f_Re, tl_restart, bpp_m3 = bcm_m3_Re)


# Write Out

## Update DFs

In [24]:
# %%capture
# Write new Parameteres to old dataframe


def get_new_params(row : pd.Series,cosmic_df : pd.DataFrame) -> tuple:
    
    """
    Gets Updated parameters
    Args:
        row (pd.Series): row of df that you want to update
        cosmic_df (pd.DataFrame): cosmic df

    Returns:
        tuple: (new a2, new e2, new i)
    """
    N = row.N

    cosmic_rows = cosmic_df[cosmic_df.N == N].reset_index(drop=True)#.query("evol_type!=10")
    
    if len(cosmic_rows)==0: 
        return [np.nan] * 50
    elif len(cosmic_rows)==1: 
        csmc_row1, csmc_row2 = cosmic_rows.loc[0], cosmic_rows.loc[0]
        csmc_row_last = cosmic_rows.loc[len(cosmic_rows)-1]
    else:
        csmc_row1, csmc_row2 = cosmic_rows.loc[0], cosmic_rows.loc[1]
        csmc_row_last = cosmic_rows.loc[len(cosmic_rows)-1]   #.to_frame().T
    c_row = csmc_row2 #second row as default

    merged_bool = False
    DWD_to_restart = cosmic_f_Re[ (cosmic_f_Re['kstar_1'].isin(WD_Types)) & (cosmic_f_Re['kstar_2'].isin(WD_Types)) ]
    DWD_to_restart = DWD_to_restart[( (DWD_to_restart['RRLO_1'] < 1) & (DWD_to_restart['RRLO_1'] < 1)) ]

    if (cosmic_rows['kstar_1'] == 15.).any() or  (cosmic_rows['kstar_2'] == 15.).any(): #(csmc_row_last.kstar_1 ==15. and csmc_row_last.kstar_2 ==15.): #if both merged
        c_row = csmc_row1
        merged_bool = True
    # elif len(DWD_to_restart) !=0:
    #     c_row = DWD_to_restart.reset_index(drop=True).loc[0]
    elif (c_row.RRLO_1 >=1) or (c_row.RRLO_2 >=1) or (c_row.stop_type == 'time'):
        c_row = csmc_row_last
    elif (c_row.ecc < 0.05) and (c_row.sep*215<0.2): #still tidally locked
        c_row = csmc_row_last
    else: #else no merger and time not finished
        c_row = csmc_row2
    
    try:
        last_nonNan_i = cosmic_rows.loc[cosmic_rows.i_new.last_valid_index()]['i_new'] #get last non-nan i for mergers
    except KeyError:
        last_nonNan_i = c_row['i_new'] 
    
    m1,m2,m3 = row.m1,row.m2,row.m3
    a1,e1,a2_old,e2_old,e2_new = row.a1,row.e1,row.a2,row.e2,c_row.e2_new
    new_a2 = c_row.a2_new
    P_out_new = np.sqrt((new_a2**3) / (c_row.mass_1 + c_row.mass_2 + c_row.m3_new) )*365.25
    
    if c_row['Restart'] == 'No':
        flag_new = 'END'
    elif c_row['Restart'] == 'Yes':
        flag_new = "C_RESTART"
        
    # h1n, h2n = get_h(m1,m2,a1,e1), get_h(m1+m2,m3,new_a2,new_e2) 
    # h1o, h2o = get_h(m1_old,m2_old,a1_old,e1_old), get_h(m1_old+m2_old,m3_old,a2_old,e2_old) 
    
    # G1n, G2n = get_G_from_h(m1,m2,h1n), get_G_from_h(m1+m2,m3,h2n)
    # G1o, G2o = get_G_from_h(m1_old,m2_old,h1o), get_G_from_h(m1_old+m2_old,m3_old,h2o)

    # Gtotn  = new_Gtot_from_old((row.m1,row.m2,row.m3),(c_row.mass_1, c_row.mass_2,row.m3),
    #                           G1o,G2o,row.i,row.e1,c_row.ecc )  
    # Gtoto = G1o*np.cos(np.deg2rad(row.i1)) + G2o*np.cos(np.deg2rad(row.i2))    
       
    # cosi = (Gtot**2 - G1n**2 - G2n**2 ) / (2*G1n*G2n)
    # cosio = (Gtoto**2 - G1o**2 - G2o**2 ) / (2*G1o*G2o)

    G1o, G1n, G2o, G2n, Gtoto, Gtotn, cosio, cosi = 0,0,0,0,0,0,0,0  #c_row.G1o, c_row.G1n, c_row.G2o, c_row.G2n, c_row.Gtoto, c_row.Gtot,c_row.cosio, c_row.cosi
    
    gamma, gamma2 = np.random.uniform(0,180), np.random.uniform(0,180)

    params = (c_row.a2_new, c_row.e2_new, last_nonNan_i,c_row.tphys*1e6, 
            int(c_row.kstar_1), int(c_row.kstar_2), 
            c_row.mass_1, c_row.mass_2, c_row.Restart, 
            c_row.rad_1* 0.00465047, c_row.rad_2* 0.00465047, c_row.ecc, c_row.sep * 0.00465047, c_row.Ad_or_Kick,
            'evol_type', c_row.porb,c_row.RRLO_1, c_row.RRLO_2, 
            'G1o', 'G1n', 'G2o', 'G2n', 'Gtoto', 'Gtotn', 'cosio', 'cosi',c_row.bin_num,c_row.m3_new, 
            c_row.type3_new, merged_bool, P_out_new, gamma, gamma2,c_row.omega_spin_1, flag_new)

    return params

def add_upd_colums(tl_df_ : pd.DataFrame, cosmic_df_ : pd.DataFrame):    

    
    new_cols = ['a2_new','e2_new','i_new','t_new','type1_new','type2_new',
                'm1_new','m2_new','m3_new','needs_Re','R1_new','R2_new','e1_new','a1_new',
                'Ad_or_Kick','evol_type','P_in_new','RRLO1','RRLO2','G1o','G1n',
                'G2o','G2n','Gtoto','Gtotn','cosi_o','cosi_n','bin_num']
    
    #tl_df_ = tl_df_.reindex(columns=list(tl_df_.columns) + new_cols, fill_value=np.nan) #create new cols
    #this process can be made into a for loop by simulatneously iterating over the new columns names/indices
    tl_df_['a2_new']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[0], axis=1)
    tl_df_['e2_new']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[1], axis=1)
    tl_df_['i_new']      = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[2], axis=1)
    tl_df_['t_new']      = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[3], axis=1)
    tl_df_['type1_new']  = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[4], axis=1)
    tl_df_['type2_new']  = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[5], axis=1)
    tl_df_['m1_new']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[6], axis=1)
    tl_df_['m2_new']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[7], axis=1)
    tl_df_['needs_Re']   = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[8], axis=1)
    tl_df_['R1_new']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[9], axis=1)
    tl_df_['R2_new']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[10], axis=1)
    tl_df_['e1_new']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[11], axis=1)
    tl_df_['a1_new']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[12], axis=1)
    tl_df_['Ad_or_Kick'] = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[13], axis=1)
    #tl_df_['evol_type']  = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[14], axis=1)
    tl_df_['P_in_new']   = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[15], axis=1)
    tl_df_['RRLO1']      = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[16], axis=1)
    tl_df_['RRLO2']      = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[17], axis=1)
    # tl_df_['G1o']        = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[18], axis=1)
    # tl_df_['G1n']        = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[19], axis=1)
    # tl_df_['G2o']        = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[20], axis=1)
    # tl_df_['G2n']        = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[21], axis=1)
    # tl_df_['Gtoto']      = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[22], axis=1)
    # tl_df_['Gtotn']      = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[23], axis=1)
    # tl_df_['cosi_o']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[24], axis=1)
    # tl_df_['cosi_n']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[25], axis=1)
    tl_df_['bin_num']    = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[26], axis=1)
    tl_df_['m3_new']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[27], axis=1)
    tl_df_['type3_new']  = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[28], axis=1)
    tl_df_['merged']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[29], axis=1)
    tl_df_['P_out_new']  = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[30], axis=1)
    tl_df_['gamma']      = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[31], axis=1)
    tl_df_['gamma2']     = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[32], axis=1)
    tl_df_['spintot_new']= tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[33], axis=1)
    tl_df_['flag_new']   = tl_df_.apply(lambda row: get_new_params(row=row,cosmic_df=cosmic_df_)[34], axis=1)
    



In [25]:
%%capture
new_cols = ['a2_new','e2_new','i_new','t_new','type1_new','type2_new',
            'm1_new','m2_new','needs_Re','R1_new','R2_new','e1_new','a1_new',
            'Ad_or_Kick','evol_type','P_in_new','RRLO1','RRLO2','G1o','G1n',
            'G2o','G2n','Gtoto','Gtotn','cosi_o','cosi_n','bin_num']

tl_restart_upd   = tl_restart.copy()
tl_NOrestart_upd = tl_no_restart.copy()

add_upd_colums(tl_restart_upd, cosmic_f_Re)
add_upd_colums(tl_NOrestart_upd, cosmic_f_noRe)


In [26]:
def update_m3_params(tl_df, bpp_m3):
    
    tl_df = tl_df.reset_index(drop=True)

    for i, row in tl_df.iterrows():
        
        this_bin_num = row['bin_num']
        bpp_rows = bpp_m3[bpp_m3['bin_num'] == this_bin_num ]
        
        if len(bpp_rows) == 0:
            continue
        else:
            last_bpp_row  = bpp_rows[-1:]
            
            tl_df.at[i,'m3_new']    = last_bpp_row['mass_2'].tolist()[0]
            tl_df.at[i,'type3_new'] = last_bpp_row['kstar_2'].tolist()[0]
            
            m1i,m2i,m3i = row.m1, row.m2, row.m3
            m1n,m2n = row.m1_new, row.m2_new
            m3n = last_bpp_row.mass_2
            
            a2i = row.a2
            a2_new = get_a_adiabatic((m1i,m2i,m3i),(m1n,m2n,m3n),a2i)
            
            tl_df.at[i,'a2_new']    = a2_new

    return tl_df


In [27]:
tl_NOrestart_upd = update_m3_params(tl_df = tl_NOrestart_upd,bpp_m3 = bpp_m3_noRe)
tl_restart_upd   = update_m3_params(tl_df = tl_restart_upd  ,bpp_m3 = bpp_m3_Re)

In [28]:
#concat both restart and no-restart dfs

Ns_1T_G = range(0,1000)
Ns_KT_G = range(1000,2000)
Ns_4    = range(4000,4500)
Ns_KT   = range(6000,7000)
Ns_1T   = range(7000,8000)

tl_df = pd.concat([tl_NOrestart_upd,tl_restart_upd])
def randomize_i(row):
    if np.isnan(row['i_new']):
        return np.random.uniform(0,180)
    else:
        return row['i_new']


## Store New Dfs Locally

In [29]:
pickle_PATH = '/Users/bijan1339/Desktop/Research/Dataframes/'

tl_df.to_pickle(pickle_PATH  + f'tl_df_{model_name}.pkl')


# ** END HERE **

In [31]:
"""
fig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(14,7))

x = np.linspace ( np.min(tl_df.a2),np.max(tl_df.a2_new) )
y = np.linspace ( np.min(tl_df.a1),np.max(tl_df.a1_new) )

ax1.plot(x,x,color='gray',linestyle='--')
ax2.plot(y,y,color='gray',linestyle='--')

ax1.scatter(tl_df.a2,tl_df.a2_new,color='limegreen',edgecolor='k')
ax2.scatter(tl_df.a1,tl_df.a1_new,color='limegreen',edgecolor='k')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax2.set_xscale('log')
ax2.set_yscale('log')

ax1.set_xlabel('a$_{2,intial}$')
ax1.set_xlabel('a$_{2,intial}$')

ax2.set_xlabel('a$_{1,intial}$')
ax2.set_xlabel('a$_{1,intial}$')
"""


"\nfig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(14,7))\n\nx = np.linspace ( np.min(tl_df.a2),np.max(tl_df.a2_new) )\ny = np.linspace ( np.min(tl_df.a1),np.max(tl_df.a1_new) )\n\nax1.plot(x,x,color='gray',linestyle='--')\nax2.plot(y,y,color='gray',linestyle='--')\n\nax1.scatter(tl_df.a2,tl_df.a2_new,color='limegreen',edgecolor='k')\nax2.scatter(tl_df.a1,tl_df.a1_new,color='limegreen',edgecolor='k')\n\nax1.set_xscale('log')\nax1.set_yscale('log')\nax2.set_xscale('log')\nax2.set_yscale('log')\n\nax1.set_xlabel('a$_{2,intial}$')\nax1.set_xlabel('a$_{2,intial}$')\n\nax2.set_xlabel('a$_{1,intial}$')\nax2.set_xlabel('a$_{1,intial}$')\n"

# Interesting Analysis TBD

1.) White Dwarf Mergers
 - Can filter Cosmic_f_XX dfs to sort by "stop_type == 'merged' and (kstar_1 in @WD_Types and kstar_2 in @WD_Types)"
 - This will give WD-WD mergers that occurred, potential 1a progenitors

2.) Unbounded WD Binaries
 - Can filter Cosmic_f_XX dfs for binaries that initially had a2<1e5 and ended with a2>1e5
 - These Triples became unbounded by the evolution of the inner binary

3.) Accelerated Stellar Evolution
 - Can find events where mass transfer accelerated the stars natural evolution



